# latex.preamble

> Deal with the preamble of a LaTeX document

In [ ]:
#| default_exp latex.preamble

In [ ]:
#| export
import re
from os import PathLike
from pathlib import Path
import warnings

In [ ]:
from trouver.helper.files_and_folders import text_from_file
from trouver.helper.tests import _test_directory

## Divide the preamble from the rest of the document

Some macros and commands defined in the preamble seem to prevent the `pylatexenc` methods from properly identifying the document environment/node in a LaTeX document. To circumvent this, we define a function to divide the preamble from the rest of the document

In [ ]:
#| export
def divide_preamble(
        text: str, # LaTeX document
        document_environment_name: str = "document"
        ) -> tuple[str, str]:
    """Divide the preamble from the rest of a LaTeX document.
    """
    begin_environment_str = rf'\begin{{{document_environment_name}}}'
    pattern = re.compile(re.escape(begin_environment_str))
    match = re.search(pattern, text) 
    start_match, end_match = match.span()
    return text[:start_match], text[start_match:]

    

In [ ]:
latex_file_path = _test_directory() / 'latex_examples' / 'example_with_a_command_with_begin.tex'
text = text_from_file(latex_file_path)

preamble, document = divide_preamble(text)
assert r'\begin{displaymath}' in preamble
assert r'Hyun Jong Kim' in preamble

assert r'Hyun Jong Kim' not in document
assert document.startswith(r'\begin{document}')
assert document.endswith('\\end{document}')

## Include `.sty` file's content into preamble

Writers often define custom commands in `.sty` files 

In [ ]:
#| export
def replace_inclusion_of_style_file_with_code(
        document: str,
        dir: PathLike # The directory containing the style file.
        ) -> str: # The modified document with style file inclusions replaced by their contents.
    r"""
    Replace style file inclusions in `document` with the code of the style files.

    This function searches for occurrences of `\usepackage{...}`, `\input{...}`, 
    `\import{...}{...}`, `\includefrom{...}{...}`, and `\subincludefrom{...}{...}`
    and replaces them with the actual contents of the corresponding `.sty` files, if available.

    """
    def replace_style_file(match):
        command = match.group(1)  # Command type
        if command in ['import', 'includefrom', 'subincludefrom']:
            path = match.group(2)
            filename = match.group(3)
            file_path = Path(dir) / path / filename
        else:
            filename = match.group(2)
            file_path = Path(dir) / filename
        # Ensure we are only processing .sty files
        if not file_path.suffix:
            file_path = file_path.with_suffix('.sty')
        
        if file_path.suffix != '.sty':
            # If it's not a .sty file, return the original command
            return match.group(0)
        # # Add .sty extension if not already present
        # if not file_path.suffix:
        #     file_path = file_path.with_suffix('.sty')
        try:
            # Read and return the contents of the style file
            with open(file_path, 'r', encoding='utf-8') as file:
                return f"% Start of included style file: {file_path.name}\n" + file.read() + f"\n% End of included style file: {file_path.name}"
        except FileNotFoundError:
            # If the file is not found, keep the original command and issue a warning
            # if file_path.suffix == '.sty':
            #     warnings.warn(f"Style file {file_path} not found. Keeping original command.", UserWarning)
            return match.group(0)

    # Regex pattern to match all relevant commands for style files
    pattern = r'\\(usepackage|input)\{([^}]+)\}|\\(import|includefrom|subincludefrom)\{([^}]+)\}\{([^}]+)\}'
    
    # Replace all matches in the document
    return re.sub(pattern, replace_style_file, document)

The  `replace_inclusion_of_style_file_with_code` function substitutes the code in style files into the appropriate locations in a latex document.

In [ ]:
latex_folder = _test_directory() / 'latex_examples' / 'latex_example_with_style_file'
main_file = latex_folder / 'main.tex'
with open(main_file, 'r', encoding='utf-8') as file:
    document = file.read()
output = replace_inclusion_of_style_file_with_code(document, latex_folder)
# preamble, body = divide_preamble(document)
# commands = custom_commands(preamble)
print(output)

\documentclass{article}

% Include a style file
% Start of included style file: mystyle.sty
% My custom style definitions
\newcommand{\mystylecommand}[1]{\textbf{#1}}

% End of included style file: mystyle.sty

\begin{document}

% Include a non-style file
\input{nonstylefile}


Hello, world! This document uses styles defined in mystyle.sty.

\end{document}



C:\Users\hyunj\AppData\Local\Temp\ipykernel_2016\2190002304.py:40: UserWarning: Style file c:\Users\hyunj\Documents\Development\Python\trouver\nbs\_tests\latex_examples\latex_example_with_style_file\nonstylefile.sty not found. Keeping original command.
  warnings.warn(f"Style file {file_path} not found. Keeping original command.", UserWarning)


It may be best practice to apply `replace_inclusion_of_styl_file_with_code` to the preamble of the document assuming that the latex document only includes style files in the preamble.

In [ ]:
latex_folder = _test_directory() / 'latex_examples' / 'latex_example_with_style_file'
main_file = latex_folder / 'main.tex'
with open(main_file, 'r', encoding='utf-8') as file:
    document = file.read()
preamble, body = divide_preamble(document)
output = replace_inclusion_of_style_file_with_code(preamble, latex_folder)
# commands = custom_commands(preamble)
print(output)

\documentclass{article}

% Include a style file
% Start of included style file: mystyle.sty
% My custom style definitions
\newcommand{\mystylecommand}[1]{\textbf{#1}}

% End of included style file: mystyle.sty


